In [59]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import statsmodels.api as sm


In [60]:
def featureDesign(features):
    features[5] = features[5]/features[6] # A ratio of free sulphur to total sulphur
    features[5] = round(features[5], 4) # Rounds the ration into 4 decimal points. Potential ERROR!!
    
    features[11] = 1.0
    features.pop(6) # TODO: Consider putting it in the main fuction
    return features
    

In [61]:
def binaryOutput(features):
    if(features[11] > 5):
        features[11] = 1.0 # One implies positive
    else:
        features[11] = 0.0
    return features

In [62]:
with open('winequality-red.csv') as wineFile:
    content = wineFile.readline()
    wine_feature_matrix = list()
    wine_output_matrix = list()
    # Reads the file line by line
    for line in wineFile:
        output = list()
        line = ''.join(line) # Creates a string
        OrigWineMatrix =line.split(';')
        
        # Removes all the white spaces 
        OrigWineMatrix = [x.strip() for x in OrigWineMatrix]
        
        # Converts all the strings into float
        OrigWineMatrix = list(map(float, OrigWineMatrix)) 
        
        # Classifies all wine qualities into positive if > 5 otherwise negative
        OrigWineMatrix = binaryOutput(OrigWineMatrix) 
        
        output.append(OrigWineMatrix[11])
        # Create a list of a list of each output i.e [1], [0], [1]
        
        wine_output_matrix.append(output) 
        #print(wine_output_matrix)
        
        wine_feature_matrix.append(featureDesign(OrigWineMatrix))
    #print(wine_matrix, type(wine_matrix))
    
    wine_output_matrix = np.array(wine_output_matrix) # Create an array of outputs
    wine_feature_matrix = np.array(wine_feature_matrix) # Create a array of features using numpy for matrix calculations
    
    #wine_output_matrix = wine_output_matrix.transpose()
    
    print(wine_output_matrix)
    print(wine_output_matrix.shape)
    print(wine_feature_matrix)
    print(wine_feature_matrix.shape)

[[0.]
 [0.]
 [0.]
 ...
 [1.]
 [0.]
 [1.]]
(1599, 1)
[[ 7.4    0.7    0.    ...  0.56   9.4    1.   ]
 [ 7.8    0.88   0.    ...  0.68   9.8    1.   ]
 [ 7.8    0.76   0.04  ...  0.65   9.8    1.   ]
 ...
 [ 6.3    0.51   0.13  ...  0.75  11.     1.   ]
 [ 5.9    0.645  0.12  ...  0.71  10.2    1.   ]
 [ 6.     0.31   0.47  ...  0.66  11.     1.   ]]
(1599, 11)


In [67]:
alpha = 0.0005 # Stepsize/learning rate
iterations = 750 # Number of times we go through the gradient descent logic

class LogisticRegression:
    def gradient_descent(self, feature_matrix, output_matrix):
        #Initialize weight_matrix to 1s
        weights_matrix = np.ones((np.shape(feature_matrix)[1], 1))
        print(weights_matrix.shape)
        for i in range(iterations):
            
            weights_matrix = weights_matrix + (alpha * feature_matrix.transpose() * (output_matrix - self.sigmoid(feature_matrix * weights_matrix)))
            
        return weights_matrix
    
    def predict(self, feature_matrix, weights_matrix):
        probs = self.logistic(sum(feature_matrix * weights_matrix))
        
        result = "Probability = " + probs.__str__() + " | classified as negative"
        
        if(probs > 0.5): result = "Probability = " + probs.__str__() + " | classified as positive"
            
        return result 
    def sigmoid(self, a): return 1.0 / (1 + np.exp(-a)) # P(y=1|x) formula
    
    #outputs the accuracy score
    def evaluate_acc(self, featuresX, true_labels, target_labels): 
        return 1

In [64]:
logisticRegression = LogisticRegression()

weights_result = logisticRegression.gradient_descent(np.mat(wine_feature_matrix), np.mat(wine_output_matrix))
print(weights_result)

logisticRegression.predict


(11, 1)
[[  1.2747123 ]
 [-17.35864715]
 [  7.13535692]
 [ -3.5774755 ]
 [ -0.8094604 ]
 [  6.93300796]
 [ -6.8858674 ]
 [-26.19537547]
 [  5.8402378 ]
 [ 14.31396183]
 [ -6.88372392]]
